In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib inline

from linalg_helper import make_random_direction, make_random_diagonal_matrix

In [2]:
def identify(A, x0, make_controller):
    controller = make_controller(x0)
    N = len(x0)
    x = x0
    for i in range(N + 1):
        direction = make_random_direction(N)
        controller.observe(direction, x.T @ direction, i)
        print(f"Estimated eigenvalues: {controller.estimate()}")
        print(f"True eigenvalues: {np.linalg.eigvals(A)}")
#         print(f"x: {x}")
        print()
        x = A @ x

In [3]:
class LeastSquaresController:
    def __init__(self, x0):
        self.N = len(x0)
        self.x0 = x0
        self.observations = []
        self.best = []
        
    class Observation:
        def __init__(self, x0, coeffs, val, order):
            self.x0 = x0
            self.coeffs = coeffs
            self.val = val
            self.order = order
            print(coeffs, val, order)
            
        def __repr__(self):
            return f"Observation({self.coeffs}, {self.val}, {self.order})"
            
        def error(self, guess):
            curr = 0
            for xi, lamb, coeff in zip(self.x0, guess, self.coeffs):
                curr += (lamb ** self.order) * xi * coeff
            return self.val - curr
    
    def observe(self, direction, proj, i):
        if i == 0:
            return
        self.observations.append(self.Observation(self.x0, direction, proj, i))
    
    def estimate(self):
        if not self.observations:
            return
        def cost(guess):
            return [observation.error(guess) for observation in self.observations]
        out = opt.least_squares(cost, np.hstack([self.best, [0]]))
        self.best = out.x
        return self.best

In [4]:
N = 10
identify(make_random_diagonal_matrix(N, 5, 0.2), 
         np.ones(N),
         LeastSquaresController)

Estimated eigenvalues: None
True eigenvalues: [4.71213092e+00 1.05364303e+00 1.94510017e-01 4.14801012e-02
 7.97213886e-03 1.53914445e-03 3.30523285e-04 6.62131614e-05
 1.26978142e-05 2.49684080e-06]

[-0.31061399  0.12646029 -0.03399101  0.41250568 -0.43873864  0.40727879
  0.52318654 -0.23077587 -0.16760108  0.05264061] -1.322625775636481 1
Estimated eigenvalues: [4.2581011]
True eigenvalues: [4.71213092e+00 1.05364303e+00 1.94510017e-01 4.14801012e-02
 7.97213886e-03 1.53914445e-03 3.30523285e-04 6.62131614e-05
 1.26978142e-05 2.49684080e-06]

[-0.25319508  0.37407848 -0.49977399  0.54798794  0.17881648 -0.3455052
 -0.07531885  0.11900426  0.20339928  0.18258512] -5.224655408425268 2
Estimated eigenvalues: [4.8206547  1.32841417]
True eigenvalues: [4.71213092e+00 1.05364303e+00 1.94510017e-01 4.14801012e-02
 7.97213886e-03 1.53914445e-03 3.30523285e-04 6.62131614e-05
 1.26978142e-05 2.49684080e-06]

[ 0.46251809 -0.27340016 -0.18199232  0.34980349 -0.43366886 -0.07714906
  0.3907865